In [1]:
NUM_PATTERN = 10
root_dir = 'lab5_pattern'

### Pastebin

In [2]:
import os
import numpy as np
np.random.seed(4242)
# # generate an integer between 0 ~ 255
# number = np.random.randint(0, 256)
# # generate a shape of 3x3 with random integer between 0 ~ 255
# vector = np.random.randint(0, 256, (3, 3))
# # generate a number between 0~2
# idx = np.random.randint(0, 3)
# # generate a number between 0~6
# number_of_action = np.random.randint(0, 6)

In [3]:
if root_dir not in os.listdir():
        os.mkdir(root_dir)

image_file_path = os.path.join(root_dir, 'image.txt')
template_file_path = os.path.join(root_dir, 'template.txt')
image_size_file_path = os.path.join(root_dir, 'image_size.txt')
action_file_path = os.path.join(root_dir, 'action.txt')
output_file_path = os.path.join(root_dir, 'output.txt')


if os.path.exists(image_file_path):
    os.remove(image_file_path)
if os.path.exists(template_file_path):
    os.remove(template_file_path)
if os.path.exists(image_size_file_path):
    os.remove(image_size_file_path)
if os.path.exists(action_file_path):
    os.remove(action_file_path)
if os.path.exists(output_file_path):
    os.remove(output_file_path)

In [4]:
def grayscale_maximum(image_size, image_R, image_G, image_B):
    '''
    find the maximum of the RGB components of each pixel
    '''
    gray_image = np.zeros((image_size, image_size),dtype=int)
    for i in range(image_size):
        for j in range(image_size):
            gray_image[i][j] = max(image_R[i][j], image_G[i][j], image_B[i][j])
    return gray_image

def grayscale_average(image_size, image_R, image_G, image_B):
    '''
    find the maximum of the RGB components of each pixel
    '''
    gray_image = np.zeros((image_size, image_size),dtype=int)
    for i in range(image_size):
        for j in range(image_size):
            gray_image[i][j] = (image_R[i][j] + image_G[i][j] + image_B[i][j]) // 3
    return gray_image

def grayscale_weighted_average(image_size, image_R, image_G, image_B):
    '''
    find the maximum of the RGB components of each pixel
    '''
    gray_image = np.zeros((image_size, image_size),dtype=int)
    for i in range(image_size):
        for j in range(image_size):
            gray_image[i][j] = image_R[i][j] // 4 + image_G[i][j] // 2 + image_B[i][j] // 4
    return gray_image

def max_pooling(image_size, gray_image):
    if image_size == 4:
        return image_size, gray_image
    else:
        new_image_size = image_size // 2
        new_gray_image = np.zeros((new_image_size, new_image_size),dtype=int)
        for i in range(new_image_size):
            for j in range(new_image_size):
                new_gray_image[i][j] = max(gray_image[2*i][2*j], gray_image[2*i+1][2*j], gray_image[2*i][2*j+1], gray_image[2*i+1][2*j+1])
        return new_image_size, new_gray_image
    
    
def negative(image_size, gray_image):
    new_gray_image = np.zeros((image_size, image_size),dtype=int)
    for i in range(image_size):
        for j in range(image_size):
            new_gray_image[i][j] = 255 - gray_image[i][j]
    return new_gray_image

def horizontal_flip(image_size, gray_image):
    new_gray_image = np.zeros((image_size, image_size),dtype=int)
    for i in range(image_size):
        for j in range(image_size):
            new_gray_image[i][j] = gray_image[i][image_size-1-j]
    return new_gray_image
        
            
def image_filter(image_size, gray_image):
    '''
    use a median filter with 3*3 mask
    '''
    new_gray_image = np.zeros((image_size, image_size),dtype=int)
    # apply replication padding
    padded_gray_image = np.pad(gray_image, ((1,1),(1,1)), 'edge')
    for i in range(image_size):
        for j in range(image_size):
            new_gray_image[i][j] = np.median(padded_gray_image[i:i+3, j:j+3])
    return padded_gray_image, new_gray_image

def cross_correlation(image_size, gray_image, template):
    '''
    1. apply zero padding to the image
    2. do convolution on the image with the 3*3 template
    '''
    new_gray_image = np.zeros((image_size, image_size),dtype=int)
    padded_gray_image = np.pad(gray_image, ((1,1),(1,1)), 'constant')
    for i in range(image_size):
        for j in range(image_size):
            new_gray_image[i][j] = np.sum(padded_gray_image[i:i+3, j:j+3] * template)
    
    return padded_gray_image, new_gray_image
    

In [5]:
# open the files with write mode
image_file = open(image_file_path, 'w')
template_file = open(template_file_path, 'w')
image_size_file = open(image_size_file_path, 'w')
action_file = open(action_file_path, 'w')
image_size_list = []

for pat in range(NUM_PATTERN):
    # write the pattern number
    image_file.write(f'{pat}\n\n')
    template_file.write(f'{pat}\n\n')
    image_size_file.write(f'{pat}\n\n')
    action_file.write(f'{pat}\n\n')
    
    image_size = np.random.choice([16])
    image_size_list.append(image_size)
    # write image size
    if (image_size == 4):
        image_size_file.write('0\n\n')
    elif (image_size == 8):
        image_size_file.write('1\n\n')
    else:
        image_size_file.write('2\n\n')
    
    # generate image for 3 channels
    image_R = np.random.randint(0, 256, (image_size, image_size))
    image_G = np.random.randint(0, 256, (image_size, image_size))
    image_B = np.random.randint(0, 256, (image_size, image_size))
    # write image
    for i in range(image_size):
        for j in range(image_size):
            image_file.write(f'{image_R[i][j]}\n')
            image_file.write(f'{image_G[i][j]}\n')
            image_file.write(f'{image_B[i][j]}\n')
    image_file.write('\n')
    
    # generate template
    template = np.random.randint(0, 256, (3, 3))
    for i in range(3):
        for j in range(3):
            template_file.write(f'{template[i][j]}\n')
    template_file.write('\n')
    
    # generate action
    action_list_set = []
    action_list = []
    for set in range(8):
        action_file.write(f'{set}\n')
        number_of_action = np.random.randint(2, 9)
        action_list = [0] * number_of_action
        action_list[0] = np.random.randint(0, 3) # grayscale action
        action_list[-1] = 7
        action_file.write(f'{len(action_list)}\n')
        for i in range(1, number_of_action-1):
            action_list[i] = np.random.randint(3, 7) # action 3~6
        for i in range(number_of_action):
            action_file.write(f'{action_list[i]}\n')
        action_file.write('\n')
        action_list_set.append(action_list)
    action_file.write('\n')

image_file.close()
template_file.close()
image_size_file.close()
action_file.close()


In [6]:
print('Image_R:\n', image_R)

Image_R:
 [[209 138 111 118 217 251  78  80 161  14   6   8 248  28 223  19]
 [172 244 103 106  79 194 229  38 180 248 230  95 145   1 207 120]
 [241 210  92  33 253 220 110  63  48  12 227 244 135 241  59  79]
 [ 77 119 177  74 153 201 115 107   7  13  97  54 163  38 188  42]
 [160  95 190 142  45 135 182 153 156 218 184 151  44 108  97 224]
 [184 135  72  19  24  74 164 163 139 240 233   3 207  26 148  79]
 [ 25 141 251  86  88 192 122 202  75   2  65  96  27  87 173 141]
 [162   6 230 193  43 127 117 149 151 192 186  36 102  51 132  94]
 [178  25 127 155  55 138 167 190 213 119 144 110 188  46  12 186]
 [ 14 141 206 179  23  65 101  28  75  35 133 143 151 193 153  65]
 [207  17 137 216 223 180 171  62  25 143  69 104 146 230  71  40]
 [ 97 224 238 215 230 154  13 235 165  97  74 176 156 211 114 127]
 [178  59 215 122 252 107 134 117 200 159  74 118 116  55 140  41]
 [165 155  15  97 238  62  83  70 235 161 208   7 232   2 129  19]
 [ 78   9   2 232 195 132 115 136 133  41  26  97  1

In [7]:
print('Template:\n', template)

Template:
 [[110  80  81]
 [ 66  11 131]
 [106  25 255]]


In [8]:
output_file = open(output_file_path, 'w')
action_file = open(action_file_path, 'r')
image_size_file = open(image_size_file_path, 'r')
image_file = open(image_file_path, 'r')
template_file = open(template_file_path, 'r')

# cacluate the result
for pat in range(NUM_PATTERN):
    output_file.write(f'{pat}\n\n')
    # read the files
    print(f'Pattern {action_file.readline().strip()} for action file')
    print(f'Pattern {image_size_file.readline().strip()} for image size file')
    print(f'Pattern {image_file.readline().strip()} for image file')
    print(f'Pattern {template_file.readline().strip()} for template file')
    
    # read image size
    image_size_file.readline()
    image_size = int(image_size_file.readline().strip())
    if (image_size == 0):
        image_size = 4
    elif (image_size == 1):
        image_size = 8
    else:
        image_size = 16
    image_size_ori = image_size
    image_size_file.readline()
    
    # read image
    image_R = np.zeros((image_size, image_size),dtype=int)
    image_G = np.zeros((image_size, image_size),dtype=int)
    image_B = np.zeros((image_size, image_size),dtype=int)
    
    image_file.readline()
    for i in range(image_size):
        for j in range(image_size):
            image_R[i][j] = int(image_file.readline().strip())
            image_G[i][j] = int(image_file.readline().strip())
            image_B[i][j] = int(image_file.readline().strip())
    image_file.readline()
    
    # read action
    action_file.readline()
    action_list_set = []
    for set in range(8):
        action_list = []
        set_number = int(action_file.readline().strip())
        number_of_action = int(action_file.readline().strip())
        for i in range(number_of_action):
            action_list.append(int(action_file.readline().strip()))
        action_file.readline()
        action_list_set.append(action_list)
    action_file.readline()
    
    # read template
    template_file.readline()
    template = np.zeros((3, 3),dtype=int)
    for i in range(3):
        for j in range(3):
            template[i][j] = int(template_file.readline().strip())
    template_file.readline()
    
    for set,action_list in enumerate(action_list_set):
        print(f'Set: {set}')
        output_file.write(f'{set}\n')
        image_size = image_size_ori
        for number, action in enumerate(action_list):
            if action == 0:
                gray_image = grayscale_maximum(image_size, image_R, image_G, image_B)
            elif action == 1:
                gray_image = grayscale_average(image_size, image_R, image_G, image_B)
            elif action == 2:
                gray_image = grayscale_weighted_average(image_size, image_R, image_G, image_B)
            elif action == 3:
                image_size, new_gray_image = max_pooling(image_size, gray_image)
            elif action == 4:
                new_gray_image = negative(image_size, gray_image)
            elif action == 5:
                new_gray_image = horizontal_flip(image_size, gray_image)
            elif action == 6:
                padded_gray_image, new_gray_image = image_filter(image_size, gray_image)
            elif action == 7:
                padded_gray_image, new_gray_image = cross_correlation(image_size, gray_image, template)
            
            print('='* 50)
            if number == 0:
                print('Converting RGB to grayscale')
                print('action:', action)
                print('gray_scale_image:\n', gray_image)
                print("image_size = ",image_size)
            else:
                print('action:', action)
                print('original_gray_scale_image:\n', gray_image)
                print('new_gray_scale_image:\n', new_gray_image)
                print("image_size = ",image_size)
                gray_image = new_gray_image
        
        # write the result in binary format
        output_file.write(f'{image_size}\n')
        for i in range(image_size):
            for j in range(image_size):
                bin_string = bin(gray_image[i][j])[2:].zfill(20)
                for k in bin_string:
                    output_file.write(f'{k}\n')
        output_file.write('\n')
        
output_file.close()
image_size_file.close()

Pattern 0 for action file
Pattern 0 for image size file
Pattern 0 for image file
Pattern 0 for template file
Set: 0
Converting RGB to grayscale
action: 2
gray_scale_image:
 [[173 139  77  99 119  67 139 137 118  81  42 128 128 124 122  90]
 [159 134 149 228 184  76 203  92 178  95 121 169 168 100 113 185]
 [ 71 129  66  63  76 135 125  72 100  80 133 133 203  90 129 123]
 [117 133  81  51 158  59 170  50 158 163 100 198 116 187 161  56]
 [164 152 131 117  80 189 151  99 180  69 110  99 179 101 129  90]
 [ 75 111 112 107 151  47 173 164 233 169  99  98 134 177 198  92]
 [120 115 117 106  78 181 133 154 164 166 156 116 146 122 189 185]
 [ 89 162 142 129 113 138   8 166 220  47  83  92 158 168  43  36]
 [221 149 107 140 138 141 143 131 107 100  88 177 177  80 125 146]
 [112 106 214  96 153 142 126  93  70 121 135 191  48 152 114 145]
 [ 76 142 136 148 123  55 154 121 127  83 125 149 156  50 104 210]
 [139 124 163 109  95 175 177  43  52 213  93 160 129 180 191 148]
 [117 162 118 158  49  